## Data Engineer Challenge


En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Introducción

Este proyecto se trata sobre procesar un archivo JSON que contiene una gran cantidad de tweets provenientes de la plataforma Twitter.

- Se deben contestar 3 incognitas:
	 <br> - Las top 10 fechas donde hay más tweets y el usuario con mas tweets en cada dia.
	 <br> - Los top 10 emojis más usados con su respectivo conteo.
	 <br> - El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

Cada una debe resolverse en 2 enfoques diferentes, uno en el que se optimice el tiempo de ejecución, y otro en que se optimice la memoria en uso.

Para optimizar el tiempo de ejecución utilice Spark por su capacidad de procesamiento distribuido y para otimizar la memoria utilize diccionarios de datos y lectura de el archivo linea por linea para evitar cargar todo el archivo en la memoria.

## Estructura del JSON

In [33]:
import pandas as pd

file_path = "farmers-protest-tweets-2021-2-4.json"
df = pd.read_json(file_path, lines=True)

In [34]:
df.head()

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,...,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24 09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,...,0,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'Kisanektamorcha', 'displayname'..."
2,https://twitter.com/parmarmaninder/status/1364...,2021-02-24 09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ...",[],[],0,0,...,0,1364506195453767680,pa,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",[https://youtu.be/-bUKumwq-J8],[https://t.co/wBPNdJdB0n],0,0,...,0,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'ReallySwara', 'displayname': 'S..."
4,https://twitter.com/KotiaPreet/status/13645061...,2021-02-24 09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre...",[],[],0,0,...,0,1364506144002088963,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None


### Modulos

In [35]:
import cProfile
import pstats
from io import StringIO
from memory_profiler import memory_usage
from collections import Counter, defaultdict
from typing import Callable, Any

## Función para medir el tiempo de ejecución

In [36]:
def time_profile_function(func: Callable[..., Any], *args: Any) -> None:
    """
    Profiles the execution of a function and prints cumulative profiling statistics.

    Args:
        func (Callable[..., Any]): The function to profile.
        *args (Any): Arguments to pass to the function.
    """
    print("Starting profiling...")
    
    # Initialize the profiler
    profiler = cProfile.Profile()
    profiler.enable()
    
    # Execute the function with provided arguments
    func(*args)
    
    # Stop the profiler
    profiler.disable()
    
    # Create a string buffer to hold the profiling data
    s = StringIO()
    # Set sorting to cumulative time
    sortby = 'cumulative'
    ps = pstats.Stats(profiler, stream=s).sort_stats(sortby)
    # Print only cumulative statistics
    ps.print_stats(5)
    
    # Output the profiling results
    print(s.getvalue())


## Función para medir la memoria utilizada

In [40]:

def profile_function(func, *args, **kwargs):
    mem_usage, return_value = memory_usage((func, args, kwargs), retval=True, max_usage=True)
    return mem_usage, return_value

# Problema 1
## Las top 10 fechas donde hay más tweets y el usuario con mas tweets en cada dia

## Descripción del Funcionamiento
### Inicialización del Diccionario:

Se utiliza un defaultdict anidado para contar el número de tweets por usuario en cada fecha. Esto permite evitar comprobaciones de existencia para claves y facilita la actualización de los conteos.

### Lectura del Archivo JSON:

Se abre el archivo JSON y se itera línea por línea para procesar cada tweet.
Se usa json.loads para convertir cada línea en un objeto JSON.

### Extracción de Datos:

La fecha del tweet se extrae y se convierte en un objeto date de datetime.
El nombre de usuario se extrae y se verifica si es válido. Si no se encuentra un nombre de usuario válido, se omite el conteo para esa línea.

### Actualización de Conteos:

El conteo de tweets se incrementa en el diccionario para la fecha y el usuario específicos.

### Ordenación y Selección:

Se ordenan las fechas por el total de tweets para cada fecha (sumando los valores en el diccionario de usuarios) y se seleccionan las 10 fechas con más tweets.

### Obtención del Usuario Principal:

Para cada una de las 10 fechas principales, se determina el usuario con el mayor número de tweets usando la función max sobre el diccionario de usuarios.

### Formato del Resultado:

Se formatea cada fecha en el formato YYYY-MM-DD y se compila en una lista de tuplas junto con el nombre del usuario con más publicaciones para esa fecha.

In [38]:
from typing import List, Tuple
from datetime import datetime
from collections import defaultdict
import json

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Get the top 10 dates with the most tweets and the username with the most posts for each day from a JSON file.
    Optimized for memory perfomance

    Args:
        file_path (str): The path to the JSON file containing the tweets.

    Returns:
        List[Tuple[datetime.date, str]]: A list of tuples where each tuple contains a date (y-m-d)
        and the username with the most posts on that date.
    """
    # Dictionary to hold the count of tweets per user per date
    date_user_counts = defaultdict(lambda: defaultdict(int))

    with open(file_path, 'r') as file:
        for line in file:
            try:
                tweet = json.loads(line)

                # Extract the tweet date and username
                date = datetime.fromisoformat(tweet['date']).date()
                username = tweet['user']['username'] if tweet['user'] and 'username' in tweet['user'] else None
                
                if date: # Only increment count if date is valid
                    # Update the tweet count for the date and user
                    if username: # Only increment count if username is valid
                        date_user_counts[date][username] += 1
            except json.JSONDecodeError:
                print("Skipping invalid JSON line.")

    # Sort dates by total tweet counts and get the top 10
    top_dates = sorted(date_user_counts.items(), key=lambda x: sum(x[1].values()), reverse=True)[:10]
    
    # For each of the top dates, find the user with the maximum number of posts
    result = [
        (date.strftime('%Y-%m-%d'), max(users.items(), key=lambda x: x[1])[0]) 
        for date, users in top_dates
    ]
    
    return result

In [ ]:
file_path = "farmers-protest-tweets-2021-2-4.json"
max_mem, result = profile_function(q1_memory, (file_path))
print(f"Memoria utilizada: {max_mem} MiB")
print(f"Resultado: {result}")



Memoria utilizada: 80.72265625 MiB
Resultado: [('2021-02-12', 'RanbirS00614606'), ('2021-02-13', 'MaanDee08215437'), ('2021-02-17', 'RaaJVinderkaur'), ('2021-02-16', 'jot__b'), ('2021-02-14', 'rebelpacifist'), ('2021-02-18', 'neetuanjle_nitu'), ('2021-02-15', 'jot__b'), ('2021-02-20', 'MangalJ23056160'), ('2021-02-23', 'Surrypuria'), ('2021-02-19', 'Preetm91')]
